In [5]:
import json
import pandas as pd

from cobra.io import read_sbml_model, write_sbml_model

from corda import CORDA
from corda.util import reaction_confidence
from fastcore import Fastcore

from fastcore import read_json, set_medium


def create_reaction_confidences(model, gene_condidence, column='CNS'):
    # Creating a reactions confidence dict using gene confidendes (df_HPAC.CNS)
    # and the corda function reaction_confidence
    rxns_confidence = dict()
    for r in model.reactions:
        # Creating a confidence dict for each particular reaction
        conf_genes = {g.id: gene_condidence[column][g.id] for g in r.genes}
        # Calculating the confidence for a reactions
        rxns_confidence[r.id] = reaction_confidence(r.gene_reaction_rule, conf_genes)
    
    return rxns_confidence

In [3]:
model = read_sbml_model('data/Recon2.2.xml')
solution = model.optimize()
print(solution.f)

with open('data/RPMI_1640_media.json') as fh:
    media_dict = json.load(fh)

with open('data/MIAPACA2_fc_rxn_penalties.json') as fh:
    fc_penalty_rxn_dict = json.load(fh)

# for k in media_dict.keys():
#     media_dict[k] = 1000

df_gene_confidences = pd.read_csv('data/genes_ConfTab.MIAPACA2_PANCREAS.tsv', sep='\t')
df_gene_confidences.set_index('hgnc_id', inplace=True)
reaction_confidences = create_reaction_confidences(model, df_gene_confidences)

set_medium(model, media_dict, inplace=True)
solution = model.optimize()
print(solution.f)

555.7858784751153
0.08444376889461447


In [6]:
core_reactions = [k for k,v in reaction_confidences.items() if v>2]
fc_builder = Fastcore(model, core_reactions, penalties=fc_penalty_rxn_dict, debug_mode=True)

blocked_reactions = fc_builder.blocked_reactions
gap_metabolites = fc_builder.gap_metabolites

Initializing Fastcore Builder using
Model: MODEL1603150001
- Nº of reactions: 7785
- Nº of core reactions: 1012
Checking network consistency (may take some minutes)
Warning, original model contains inconsistent reactions
- Nº of blocked reactions: 2975
- Nº of gap metabolites: 2740

Pruning model inconsistencies
- Nº of reactions after pruning: 4810
- Nº of core reactions after pruning: 713

Initializing LP7 and LP10
Fastcore builder ready!


In [ ]:
fc_builder._scaling_factor = 1e5
fc_builder.fast_core()
# fastcore_model = fc_builder.build_context_specific_model()
fastcore_rxns_4  = fc_builder.get_context_specific_set()

In [ ]:
# corda_builder = CORDA(model, reaction_confidences)
# corda_builder.build()
# corda_model = corda_builder.cobra_model()
# fastcore_rxns = {r.id for r in cs_model.reactions}
# corda_blocked = Fastcore.fast_find_blocked(corda_model)

In [ ]:
corda_rxns = {r.id for r in corda_model.reactions}
for i in range(-1,4):
    fastcore_count = len([r for r in fastcore_rxns if reaction_confidences[r] == i])
    corda_count = len([r for r in (corda_rxns - blocked) if reaction_confidences[r] == i])
    total_count = len([r.id for r in model.reactions if reaction_confidences[r.id] == i])
    print(i, fastcore_count, corda_count, total_count)

print()
jaccard = len((corda_rxns - corda_blocked) & fastcore_rxns) / len(corda_rxns | fastcore_rxns)
print(jaccard)

In [ ]:
# fastcore_model = fc_builder.build_context_specific_model()
# corda_blocked = Fastcore.fast_find_blocked(fastcore_model)
# corda_blocked

len(fastcore_rxns_5 & fastcore_rxns_4) / len(fastcore_rxns_5 | fastcore_rxns_4) 

In [7]:
model.metabolites.xolest2_hs_c.name


'cholesterol ester'